In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [3]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [4]:
def objective(params):
    return train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )

In [5]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [6]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/05/24 12:03:51 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 19s 434ms/step - loss: 30.1236 - root_mean_squared_error: 5.4885
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.8413 - root_mean_squared_error: 2.2774 - val_loss: 0.6395 - val_root_mean_squared_error: 0.7997

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.5582 - root_mean_squared_error: 0.7471
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6037 - root_mean_squared_error: 0.7769 - val_loss: 0.5665 - val_root_mean_squared_error: 0.7527

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.4991 - root_mean_squared_error: 0.7065
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5376 - root_mean_squared_error: 0.7331 - val_loss: 0.5269 - val_root_mean_squared_error: 0.7259

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.5084 - root_mean_squared_error: 0.7130
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/s

c:\git\mlflowtest\.venv\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

2024/05/24 12:03:58 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 371ms/step - loss: 39.3067 - root_mean_squared_error: 6.2695
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 31.8418 - root_mean_squared_error: 5.6285 - val_loss: 13.5595 - val_root_mean_squared_error: 3.6823

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 14.7112 - root_mean_squared_error: 3.8355
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.5888 - root_mean_squared_error: 3.2452 - val_loss: 4.6931 - val_root_mean_squared_error: 2.1664

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 5.3469 - root_mean_squared_error: 2.3123
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9961 - root_mean_squared_error: 1.9964 - val_loss: 2.6175 - val_root_mean_squared_error: 1.6179

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - los

2024/05/24 12:04:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 17s 381ms/step - loss: 33.9528 - root_mean_squared_error: 5.8269
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 26.6264 - root_mean_squared_error: 5.1353 - val_loss: 6.8467 - val_root_mean_squared_error: 2.6166

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 6.7997 - root_mean_squared_error: 2.6076
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.9618 - root_mean_squared_error: 2.2194 - val_loss: 2.6442 - val_root_mean_squared_error: 1.6261

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.6797 - root_mean_squared_error: 1.6370
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3974 - root_mean_squared_error: 1.5472 - val_loss: 2.1610 - val_root_mean_squared_error: 1.4700

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 

2024/05/24 12:04:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



100%|██████████| 8/8 [00:45<00:00,  5.71s/trial, best loss: 0.7258833646774292]


2024/05/24 12:04:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Best parameters: {'lr': 0.05415039628619479, 'momentum': 0.6879686636866701}
Best eval rmse: 0.7258833646774292


# Tilbake til UI

Vi kan sjekke UI-en for å se hvordan det gikk.
Her kan vi registrere den beste modellen.
```bash
    mlflow ui
```

Etter at vi har registrert modellen kan vi serve den slik:

```bash
    mlflow models serve -m "models:/WineQuality/1" --port 5002
```

( må installere 
```ps
    Invoke-WebRequest -UseBasicParsing -Uri "https://raw.githubusercontent.com/pyenv-win/pyenv-win/master/pyenv-win/install-pyenv-win.ps1" -OutFile "./install-pyenv-win.ps1"; &"./install-pyenv-win.ps1"
```
)


# Test av served modell

In [8]:
import requests

url = 'http://localhost:5002/invocations'
headers = {'Content-Type': 'application/json'}
data = {
    "dataframe_split": {
       "columns": [
            "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
            "chlorides", "free sulfur dioxide", "total sulfur dioxide",
            "density", "pH", "sulphates", "alcohol"
        ],
        "data": [
            [7, 0.27, 0.36, 20.7, 0.045, 45, 170, 1.001, 3, 0.45, 8.8]
        ]
    }
}

response = requests.post(url, json=data, headers=headers)
print(response.text)


{"error_code": "BAD_REQUEST", "message": "Invalid input. Provided dataframe_split field is not a valid dataframe representation in 'split' format. Error: '4 columns passed, passed data had 11 columns'"}
